# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [18]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json


# Import API key
from api_keys import geoapify_key

In [19]:
# Load the CSV file created in Part 1 into a Pandas DataFrame3

city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,punta arenas,-53.1500,-70.9167,8.01,81,66,12.28,CL,1667066380
1,1,kapaa,22.0752,-159.3190,24.99,90,7,0.45,US,1667066238
2,2,ancud,-41.8697,-73.8203,9.88,57,65,9.25,CL,1667066380
3,3,vaini,-21.2000,-175.2000,21.09,94,40,1.54,TO,1667066381
4,4,ushuaia,-54.8000,-68.3000,2.81,93,100,10.80,AR,1667066219


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [20]:
%%capture --no-display
map_url= 'https://maps.geoapify.com/v1/styles/osm-carto/style.json?'

fig1 = map_url + 'apiKey=' + geoapify_key


# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE
fig1

'https://maps.geoapify.com/v1/styles/osm-carto/style.json?apiKey=397ec048ee054d66a6368b1272c91532'

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [21]:
# Narrow down cities that fit criteria and drop any results with null values
par_1 = city_data_df.loc[(city_data_df['Cloudiness'] < 10)]

ideal_city_df = par_1.loc[(par_1["Max Temp"] > 27)]

# Drop any rows with null values
ideal_city_df= ideal_city_df[["City", "Country", "Lat", "Lng", "Humidity"]]
ideal_city_df = ideal_city_df.dropna()

# Display sample data
ideal_city_df

,City,Country,Lat,Lng,Humidity
24,chuy,UY,-33.6971,-53.4616,38
62,progreso,MX,21.2833,-89.6667,59
106,praia,CV,14.9215,-23.5087,20
123,leo,BF,11.1000,-2.1000,25
159,kavaratti,IN,10.5669,72.6420,64
179,kahului,US,20.8947,-156.4700,84
195,ixtapa,MX,20.7000,-105.2000,64
210,tartagal,AR,-22.5164,-63.8013,16
290,taoudenni,ML,22.6783,-3.9836,8
325,yuriria,MX,20.2000,-101.1500,33


### Step 3: Create a new DataFrame called `hotel_df`.

In [32]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# hotel_df = ideal_city_df.copy()
hotel_df = ideal_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
24,chuy,UY,-33.6971,-53.4616,38,
62,progreso,MX,21.2833,-89.6667,59,
106,praia,CV,14.9215,-23.5087,20,
123,leo,BF,11.1000,-2.1000,25,
159,kavaratti,IN,10.5669,72.6420,64,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [35]:
## Checking my API key works

# url = 'https://api.geoapify.com/v2/places?categories=catering.restaurant,catering.cafe&filter=rect:11.573106549898483,48.13898913611139,11.57704581350751,48.13666585409989&limit=20&apiKey='

# response_address = requests.get(url + geoapify_key)
# name_address = response_address.json()
# print(json.dumps(name_address, indent=4, sort_keys=True))


In [34]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'filter=circle:{lat},{lng},{radius}'
    params["bias"] = f'bias=proximity:{lat},{lng},'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
chuy - nearest hotel: No hotel found
progreso - nearest hotel: No hotel found
praia - nearest hotel: No hotel found
leo - nearest hotel: No hotel found
kavaratti - nearest hotel: No hotel found
kahului - nearest hotel: No hotel found
ixtapa - nearest hotel: No hotel found
tartagal - nearest hotel: No hotel found
taoudenni - nearest hotel: No hotel found
yuriria - nearest hotel: No hotel found
rio claro - nearest hotel: No hotel found
santiago del estero - nearest hotel: No hotel found
djenne - nearest hotel: No hotel found
corralillo - nearest hotel: No hotel found
araouane - nearest hotel: No hotel found
sal rei - nearest hotel: No hotel found
parana - nearest hotel: No hotel found
filingue - nearest hotel: No hotel found
buin - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
24,chuy,UY,-33.6971,-53.4616,38,No hotel found
62,progreso,MX,21.2833,-89.6667,59,No hotel found
106,praia,CV,14.9215,-23.5087,20,No hotel found
123,leo,BF,11.1000,-2.1000,25,No hotel found
159,kavaratti,IN,10.5669,72.6420,64,No hotel found
179,kahului,US,20.8947,-156.4700,84,No hotel found
195,ixtapa,MX,20.7000,-105.2000,64,No hotel found
210,tartagal,AR,-22.5164,-63.8013,16,No hotel found
290,taoudenni,ML,22.6783,-3.9836,8,No hotel found
325,yuriria,MX,20.2000,-101.1500,33,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE